In [1]:
#!pip install -U ipython
#!pip install py2neo
#!pip install pandas

In [2]:
import py2neo
import pandas as pd

In [3]:
from py2neo import Database,Graph,Node,Relationship

In [4]:
db = Database()
db

<Database uri='bolt://localhost:7687' secure=False user_agent='py2neo/4.3.0 neobolt/1.7.17 Python/3.8.2-final-0 (win32)'>

In [10]:
ml = Graph("bolt://localhost:7687",user="neo4j2",password="neo4j")
tx=ml.begin()

In [6]:
df_peliculas=pd.read_csv('./movies_enriched.csv', sep=',')
df_peliculas

,Unnamed: 0,movieId,title,dbpedia_link,genres,director_uri,director,country_uri,country,runtime,writer_uri,writer,starring,languagues,studios
0,0,781,Stealing Beauty (1996),http://dbpedia.org/resource/Stealing_Beauty,Drama,http://dbpedia.org/resource/Bernardo_Bertolucci,Bernardo Bertolucci,http://dbpedia.org/resource/France,France,7140.0,http://dbpedia.org/resource/Susan_Minot,Susan Minot,Sinéad Cusack|Joseph Fiennes|Rachel Weisz|Jere...,French language|German language|Italian langua...,Recorded Picture Company|Fox Searchlight Pictu...
1,1,1799,Suicide Kings (1997),http://dbpedia.org/resource/Suicide_Kings,Comedy|Crime|Drama|Mystery|Thriller,http://dbpedia.org/resource/Peter_O'Fallon,Peter O'Fallon,NaN,NaN,6360.0,NaN,NaN,Henry Thomas|Denis Leary|Christopher Walken|Ja...,NaN,NaN
2,2,521,Romeo Is Bleeding (1993),http://dbpedia.org/resource/Romeo_Is_Bleeding,Crime|Thriller,http://dbpedia.org/resource/Peter_Medak,Peter Medak,NaN,NaN,6600.0,http://dbpedia.org/resource/Hilary_Henkin,Hilary Henkin,Roy Scheider|Annabella Sciorra|Lena Olin|Gary ...,NaN,PolyGram Filmed Entertainment|Working Title Films
3,3,3596,Screwed (2000),http://dbpedia.org/resource/Screwed_(2000_film),Comedy,http://dbpedia.org/resource/Scott_Alexander_an...,Scott Alexander and Larry Karaszewski,NaN,NaN,4860.0,NaN,NaN,Sarah Silverman|Danny DeVito|Norm Macdonald|El...,NaN,Brillstein-Grey Entertainment
4,4,3682,Magnum Force (1973),http://dbpedia.org/resource/Magnum_Force,Action|Crime|Drama|Thriller,http://dbpedia.org/resource/Clint_Eastwood,Clint Eastwood,NaN,NaN,7440.0,http://dbpedia.org/resource/Michael_Cimino,Michael Cimino,Clint Eastwood|Robert Urich|David Soul|Tim Mat...,NaN,The Malpaso Company
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282,3282,1013,"Parent Trap, The (1961)",http://dbpedia.org/resource/The_Parent_Trap_(1...,Children|Comedy|Romance,http://dbpedia.org/resource/David_Swift_(direc...,David Swift (director),NaN,NaN,7680.0,http://dbpedia.org/resource/David_Swift_(direc...,David Swift (director),Hayley Mills|Brian Keith|Maureen O'Hara,NaN,Walt Disney Pictures
3283,3283,2634,"Mummy, The (1959)",http://dbpedia.org/resource/The_Mummy_(1959_film),Horror,http://dbpedia.org/resource/Terence_Fisher,Terence Fisher,NaN,NaN,5280.0,http://dbpedia.org/resource/Jimmy_Sangster,Jimmy Sangster,Christopher Lee|George Pastell|Peter Cushing|E...,NaN,Hammer Film Productions
3284,3284,2242,"Grandview, U.S.A. (1984)","http://dbpedia.org/resource/Grandview,_U.S.A.",Drama,http://dbpedia.org/resource/Randal_Kleiser,Randal Kleiser,NaN,NaN,5820.0,http://dbpedia.org/resource/Ken_Hixon,Ken Hixon,Troy Donahue|Jamie Lee Curtis|C. Thomas Howell...,NaN,CBS Theatrical Films
3285,3285,3941,Sorority House Massacre (1986),http://dbpedia.org/resource/Sorority_House_Mas...,Horror,NaN,NaN,http://dbpedia.org/resource/United_States,United States,4560.0,NaN,NaN,NaN,NaN,NaN


In [7]:
ml.schema.create_uniqueness_constraint("Movie","movieId")
ml.schema.create_uniqueness_constraint("Country","countryUri")
ml.schema.create_uniqueness_constraint("Director","directorUri")
ml.schema.create_uniqueness_constraint("Writer","writerUri")
ml.schema.create_uniqueness_constraint("Gender","gender")
ml.schema.create_uniqueness_constraint("Language","language")
ml.schema.create_uniqueness_constraint("Studio","studio")
ml.schema.create_uniqueness_constraint("Actor","actor")

for i, row in df_peliculas.iterrows():
    if(df_peliculas["runtime"].values[i]!=null):
        m=Node("Movie",movieId=df_peliculas["movieId"].values[i],title=df_peliculas["title"].values[i],dbpediaLink=df_peliculas["dbpedia_link"].values[i],runtime=str(int(df_peliculas["runtime"].values[i])/60))
    else:
        m=Node("Movie",movieId=df_peliculas["movieId"].values[i],title=df_peliculas["title"].values[i],dbpediaLink=df_peliculas["dbpedia_link"].values[i])

    tx.merge(m,"Movie","movieId")

    if(df_peliculas["director_uri"].values[i]!=null):
        d=Node("Director",directorUri=df_peliculas["director_uri"].values[i],director=df_peliculas["director"].values[i])
        tx.merge(d,"Director","director_uri")
    if(df_peliculas["writer_uri"].values[i]!=null):
        w=Node("Writer",writerUri=df_peliculas["writer_uri"].values[i],writer=df_peliculas["writer"].values[i])
        tx.merge(w,"Writer","writer_uri")
    if(df_peliculas["country_uri"].values[i]!=null):
        c=Node("Country",countryUri=df_peliculas["country_uri"].values[i],country=df_peliculas["country"].values[i])
        tx.merge(c,"Country","country_uri")
        if(df_peliculas["genres"].values[i]!=null):
            for ge in df_peliculas["genres"].values[i].split("|"):
                g=Node("Gender",gender=ge)
                tx.merge(g,"Gender","gender")
        if(df_peliculas["starring"].values[i]!=null):
            for ac in df_peliculas["starring"].values[i].split("|"):
                a=Node("Actor",actor=ac)
                tx.merge(a,"Actor","actor")
        if(df_peliculas["languagues"].values[i]!=null):
            for la in df_peliculas["languagues"].values[i].split("|"):
                l=Node("Language",language=la)
                tx.merge(l,"Language","language")
        if(df_peliculas["studios"].values[i]!=null):
            for st in df_peliculas["studios"].values[i].split("|"):
                stu=Node("Studio",studio=st)
                tx.merge(stu,"Studio","studio")
    
    if(d!=null):
        dm = Relationship(d, "DIRECTS", m)
        tx.merge(dm)
    if(w!=null):
        wm = Relationship(w, "WRITES", m)
        tx.merge(wm)
    if(c!=null):
        mc = Relationship(m, "FILMED", c)
        tx.merge(mc) 
    
    tx.commit()

ValueError: Primary label and primary key are required for MERGE operation

In [13]:
 ml.run("MATCH (m:Movie) RETURN m.title").to_table()

m.title
Newsies (1992)
Exotica (1994)
"Gay Divorcee, The (1934)"
City Hall (1996)
"Simple Plan, A (1998)"
Heavy (1995)
"Last Time I Committed Suicide, The (1997)"
True Lies (1994)
Light of Day (1987)
Exit to Eden (1994)


In [9]:
len(ml.nodes)

27239